In [36]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
! pip install folium==0.5.0
import folium 
import matplotlib.cm as cm
import matplotlib.colors as colors

print('import successful')

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
import successful


In [37]:
#bring in previous df
df=pd.read_csv('Capstone2.csv')
df.head()

,Unnamed: 0,Unnamed: 0.1,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,0,0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,1,1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,2,2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,3,3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,4,4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [38]:
#borough only in Toronto
dft=df[df['Borough'].str.contains('Toronto')]
dfto=dft.reset_index(drop=True)
dfto.head()

,Unnamed: 0,Unnamed: 0.1,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,37,37,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,41,41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,42,42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,43,43,M4M,East Toronto,Studio District,43.659526,-79.340923
4,44,44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [39]:
dfto.shape

(39, 7)

In [40]:
#remove 2 unnamed columns to declutter
col_list=['PostalCode','Borough','Neighborhood','Latitude','Longitude']
df=dfto[col_list]
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [51]:
#list all borough options
df['Borough'].value_counts()

Downtown Toronto    19
Central Toronto      9
West Toronto         6
East Toronto         5
Name: Borough, dtype: int64

In [52]:
#create integer value for map
df['section']=df['Borough'].replace(to_replace=['Downtown Toronto','Central Toronto','West Toronto','East Toronto'],value=[1,2,3,4],inplace=False)
df.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


,PostalCode,Borough,Neighborhood,Latitude,Longitude,section
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,4
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,4
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,4
3,M4M,East Toronto,Studio District,43.659526,-79.340923,4
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2


In [53]:
#toronto coordinates
address = 'Toronto'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(f'The geograpical coordinate of Toronto are {latitude}, {longitude}.')

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [55]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)
kclusters=5
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(df['Latitude'], df['Longitude'], df['Neighborhood'], df['section']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

In [57]:
map_clusters